# Creating the U.S. Flag

We start off by setting up the basic dimensions for the flag. 

In [2]:
stripe_width = 100
flag_height = 13 * stripe_width
flag_width = 1.9 * flag_height

print(flag_width, flag_height)

2470.0 1300


We will use these dimensions to set the drawing canvas. We begin our SVG file by creating a field of stripes:

In [3]:
svg1 = """
<svg xmlns="http://www.w3.org/2000/svg" width="%d" height="%d">
""" % (flag_width, flag_height)
svg2 = """
</svg>
""" 
print(svg1)
print(svg2)


<svg xmlns="http://www.w3.org/2000/svg" width="2470" height="1300">


</svg>



We will build the basic flag by creating a red rectangle on which we will draw the required white stripes as very thick lines.

In [4]:
flag = """
  <rect width="%d" height="%d" 
  style="fill:rgb(255,0,0)" />""" % (flag_width, flag_height)

print(flag)


  <rect width="2470" height="1300" 
  style="fill:rgb(255,0,0)" />


Here is the code for a fat white line:

In [5]:
line = """<line x="%d" y="%d" style="stroke:rgb(255,255,255);stroke-width:%d" />""" % (0, flag_width, stripe_width)

print(line)

<line x="0" y="2470" style="stroke:rgb(255,255,255);stroke-width:100" />


The star field is another blue rectangle:

In [6]:
star_field = """
<rect width="%d" height="%d"
style="fill:rgb(0,0,255)" />
""" % (flag_height * 0.76, 7 * stripe_width) 

print(star_field)


<rect width="988" height="700"
style="fill:rgb(0,0,255)" />



We now have the pieces needed to build the basic flag image (neglecting the star field for now):

In [7]:
def build_flag(width, height):
    svg1 = """
<svg xmlns="http://www.w3.org/2000/svg" width="%d" height="%d">
""" % (flag_width, flag_height)
    svg2 = """
</svg>
"""
    flag = """
  <rect width="%d" height="%d" 
  style="fill:rgb(255,0,0)" />""" % (flag_width, flag_height)
    star_field = """
<rect width="%d" height="%d"
style="fill:rgb(0,0,255)" />""" % (flag_height * 0.76, 7 * flag_height / 13)
    svg = svg1
    svg += "  <g>"
    svg += flag
    x1 = 0;
    x2 = width
    stripe_width = height/13
    y = 1.5 * stripe_width
    for stripe in range(0,7):
        line = """<line x1="%d" y1="%d" x2="%d" y2="%d" 
  style="stroke:rgb(255,255,255);stroke-width:%d" />""" % (0, y, width, y, stripe_width )
        svg += line
        y += 2 * stripe_width
    svg += star_field
    svg += "  </g>"
    svg += svg2
    with open('_images/us-flag.svg','w') as fout:
        fout.write(svg)

To see the results of calling this function, we need the **show** routine from the presious notebook

In [8]:
def show(fname):
    with open(fname) as fin:
        lines = fin.readlines()
        for l in lines:
            print(l.rstrip())

In [9]:
build_flag(flag_width, flag_height)
show('_images/us-flag.svg')


<svg xmlns="http://www.w3.org/2000/svg" width="2470" height="1300">
  <g>
  <rect width="2470" height="1300"
  style="fill:rgb(255,0,0)" /><line x1="0" y1="150" x2="2470" y2="150"
  style="stroke:rgb(255,255,255);stroke-width:100" /><line x1="0" y1="350" x2="2470" y2="350"
  style="stroke:rgb(255,255,255);stroke-width:100" /><line x1="0" y1="550" x2="2470" y2="550"
  style="stroke:rgb(255,255,255);stroke-width:100" /><line x1="0" y1="750" x2="2470" y2="750"
  style="stroke:rgb(255,255,255);stroke-width:100" /><line x1="0" y1="950" x2="2470" y2="950"
  style="stroke:rgb(255,255,255);stroke-width:100" /><line x1="0" y1="1150" x2="2470" y2="1150"
  style="stroke:rgb(255,255,255);stroke-width:100" /><line x1="0" y1="1350" x2="2470" y2="1350"
  style="stroke:rgb(255,255,255);stroke-width:100" />
<rect width="988" height="700"
style="fill:rgb(0,0,255)" />  </g>
</svg>


Here is the flag so far:

<img src="_images/us-flag.svg">

## Creating the Star Field

While searching for a flag SVG, I came across a clever way to build the star field. Basically, we create two vertical sets of stars, then group them in the needed pattern. This is very efficient, but takes some study to figure out. 

SVG makes building a star easy, since it will fill any closed polygon, even one that crosses itself.

In [10]:
stars = """
<g fill="#fff">
  <g id="s18">
    <g id="s9">
      <g id="s5">
        <g id="s4">
          <!-- here is the path for a single star -->
          <path id="s"
            d="
              M247,90
              317.534230,307.082039
              132.873218,172.917961
              H361.126782
              L176.465770,307.082039
              z"/>
            <use xlink:href="#s" y="420"/>
            <use xlink:href="#s" y="840"/>
            <use xlink:href="#s" y="1260"/>
        </g>
        <use xlink:href="#s" y="1680"/>
      </g>
      <use xlink:href="#s4" x="247" y="210"/>
    </g>
    <use xlink:href="#s9" x="494"/>
  </g>
  <use xlink:href="#s18" x="988"/>
  <use xlink:href="#s9" x="1976"/>
  <use xlink:href="#s5" x="2470"/>
</g>
"""

## Clipping with a heart shape

The flag is clipped using a heart shape path. we will build a simple shape using a square and two circular arcs. Before setting this up, we need to do some calculastions.

The total height of our heart needs to match the height of the flag. Since we will be rotating the square so it lines up properly, we need a bit of math to figure out how bit go make the sides.

We draw the heart by setting up a square, and capping two adjacent edges with circular arcs. This is easy to do with SVG, but calculating the size of the box takes a bit of math.

We will define the path with the square sitting at the top of the screen. The circluar arcs will be on the right and bottom edges.

The radius of the circluar ac is one half the size of the side. The hard part is figuring out the size of the side so the resulting heart shape is exactly the height of the flag.

Here is some code:

flag_height = 1300

The height of the heart is given by:

height = side * cos(45) * 1.5 + side/2

height = side * (1.5 * cos(45) + 0.5)

= 1.56 * side. 

side = height / 1.56

Let's see what this gives:

In [11]:
import math
flag_height = 3900
fac = 1.5 * math.cos(45 * math.pi/180.0) + 0.5
side = flag_height / fac

print(fac, side, side, side / 2)

1.5606601717798214 2498.9424799329176 2498.9424799329176 1249.4712399664588


Using the flag code found on Wikipedia, I calculated the size of the heart shape and added the clip path. Here is the final code for the flag, clipped to the heart shape.

In [12]:
show("_images/usflag.svg")

<svg
    xmlns="http://www.w3.org/2000/svg"
    xmlns:xlink="http://www.w3.org/1999/xlink"
    width="1235" height="650"
    viewBox="0 0 7410 3900">
  <defs>
    <clipPath id="heart">
      <path
        transform="
          translate(2964,3900)
          rotate(225)
        "
        d="M 0,2499 V 0 h 2499 a 1249,1249 90 0 1 0,2499 1249,1249 90 0 1 -2499,0 z"
        style="fill:none" stroke="red" stroke-width="5" />
    </clipPath>
  </defs>
  <g clip-path="url(#heart)" >
    <!-- create a basic red rectangle -->
    <rect width="7410" height="3900" fill="#b22234"/>
    <!-- draw wide lines to create the white stripes -->
    <path d="
      M0,450
      H7410
      m0,600
      H0
      m0,600
      H7410
      m0,600
      H0m0,600
      H7410
      m0,600
      H0"
      stroke="#fff"
      stroke-width="300"/>
    <!-- add the blue star background -->
    <rect width="2964" height="2100" fill="#3c3b6e"/>
    <!-- this code creates the star field in a clever nested set of groups

And here is the result:
    
<img src="_images/usflag.svg">

## Building a star

The following material is the result of testing SVG code. Since the image above is all we need for the club logo, this material is here for reference only

The hardest part of generating the flag is placing all of the stars. Let's start off by building one star.

In [13]:
def star(diameter):
    svg = """
<svg xmlns="http://www.w3.org/2000/svg" width="%d" height="%d">
""" % (diameter, diameter)
    radius = diameter/2
    x = radius
    y = radius
    circle = """
  <circle cx="%d" cy="%d" r="%d" style="fill:none;stroke:rgb(0,0,0);stroke-width:2" />
""" % (x,y,radius)
    svg += circle
    svg += """</svg>"""
    with open('_images/us-star.svg','w') as fout:
        fout.write(svg)
        
star(100)
show('_images/us-star.svg')


<svg xmlns="http://www.w3.org/2000/svg" width="100" height="100">

  <circle cx="50" cy="50" r="50" style="fill:none;stroke:rgb(0,0,0);stroke-width:2" />
</svg>


We should see a simple circle.

<img src="_images/us-star.svg">

Now for the fun part. We need to calculate the points for the star. These appear at 72 degree intervals from the top. A little trig will get those points set up:



In [14]:
import math
angle = 0
radius = 50

points = []
for p in range(5):
    x = radius + radius * math.sin(angle * math.pi / 180.0)
    y = radius - radius * math.cos(angle * math.pi / 180.0)
    sx = '%.2f' % x
    sy = '%.2f' % y
    points.append([sx,sy])
    angle += 72
print(points)

[['50.00', '0.00'], ['97.55', '34.55'], ['79.39', '90.45'], ['20.61', '90.45'], ['2.45', '34.55']]


Let's draw a path using these points. SVG has an interesting trick we can use to create the star. If we draw a **polyline** using these points in the correct order, we will get the star.

the correct order is this list (starting with zero at the top):

In [68]:
order = [0,2,4,1,3]

In [65]:
path = """
<polygon  points="
"""
  %s, %s % (points[0][0], points[0][1])
for p in points:
    x = p[0]
    y = p[1]
    path += """
  %s %s""" % (x,y)
path += """ Z" style="fill:#000000" />""" 
    
print(path)


<path d="M 50.00 0.00
  L
  50.00 0.00
  97.55 34.55
  79.39 90.45
  20.61 90.45
  2.45 34.55 Z" style="fill:#000000" />


In [70]:
svg = """
<polygon points = "
"""
for p in order:
    px = points[p][0]
    py = points[p][1]
    svg += "  %s,%s" % (px, py)
svg += "\""
svg += """style="fill:white; rule:nonzero;" />
"""

print(svg)


<polygon points = "
  50.00,0.00  79.39,90.45  2.45,34.55  97.55,34.55  20.61,90.45"style="fill:white; rule:nonzero;" />



We now have code we can use to generate a white star for placement on the star field

In [79]:
def gen_star(diameter, color):
    svg = """
<svg xmlns="http://www.w3.org/2000/svg" width="%d" height="%d">   
    """ % (diameter, diameter)
    radius = diameter / 2
    angle = 0
    points = []
    for p in range(5):
        x = radius + radius * math.sin(angle * math.pi / 180.0)
        y = radius - radius * math.cos(angle * math.pi / 180.0)
        sx = '%.2f' % x
        sy = '%.2f' % y
        points.append([sx,sy])
        angle += 72
    order = [0,2,4,1,3]
    svg += """
<polygon points = "
"""
    for p in order:
        px = points[p][0]
        py = points[p][1]
        svg += "  %s,%s" % (px, py)
    svg += "\""
    svg += """ style="fill:%s; rule:nonzero;" />
""" % color
    svg += "</svg>"
    with open('_images/haffa-star.svg','w') as fout:
        fout.write(svg)

Now we can display this star

In [80]:
gen_star(100, "red")

In [81]:
show('_images/haffa-star.svg')


<svg xmlns="http://www.w3.org/2000/svg" width="100" height="100">

<polygon points = "
  50.00,0.00  79.39,90.45  2.45,34.55  97.55,34.55  20.61,90.45" style="fill:red; rule:nonzero;" />
</svg>


And, here is the test star.

<img src="_images/haffa-star.svg">

In [ ]:
We have all the pieces needed to build the final logo, we will do that in the next article.